# First, extract the frames from the video
Code is based on StackOverflow:
https://stackoverflow.com/questions/33311153/python-extracting-and-saving-video-frames

In [47]:
import cv2

name = "victor"
croppedFolder = name + "/cropped/"

if not os.path.exists(name + "/"):
    os.makedirs(name + "/")

if not os.path.exists(croppedFolder):
    os.makedirs(croppedFolder)


vidcap = cv2.VideoCapture(name + '.mp4')
success,image = vidcap.read()
count = 0
while success:
    cv2.imwrite(name + "/frame%d.jpg" % count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    count += 1
print("Finished! Fully finished: " + str(success))

Finished! Fully finished: False


# Setup model

In [48]:
# %tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

2.10.0


In [49]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

# Importing libraries

In [50]:
import os
import cv2
import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
import sys
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Helper functions

In [51]:
def findRange(box):
    # Coords in {TopDist, LeftDist, BotDist, RightDist}
    # Returns a value between -1 and 1 for left and right
    leftDist = box['box'][1]
    rightDist = box['box'][3]
    avg = (rightDist - leftDist) / 2 + (leftDist)
    return avg * 2 - 1
    
def reduceBoxes(boxes, scores, classes):
    '''Function to reduce all the boxes to only the ones being displayed.
    Slightly modified version of Shreyas Vedpathak's version from stackoverflow:
    https://stackoverflow.com/a/68376832/19442761'''
    
    max_boxes_to_draw = boxes.shape[0]
    min_score_thresh=.5
    
    # Loop over all objects
    coordinates = []
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores[0][i] > min_score_thresh:
            class_id = int(classes[0][i] + 1)
            coordinates.append({
                "box": boxes[i],
                "score": scores[0][i]
            })

    return coordinates

def cropImg(image, box):
    # box in {TopDist, LeftDist, BotDist, RightDist}, in ratios
    # left, upper, right, and lower
    PIL_image = Image.fromarray(np.uint8(image)).convert('RGB')
    siz = PIL_image.size
    hwhw = np.tile(siz, 2).flatten()

    # Cropping as left, upper, right, and lower
    newbox = np.array([box[1], box[0], box[3], box[2]])
    imgcoords = hwhw * newbox
 
    return PIL_image.crop(imgcoords)

# Setting up and loading the model

In [52]:
%matplotlib inline

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = 'weights/frozen_inference_graph.pb'

# Path to label map file
PATH_TO_LABELS = 'weights/label_map.pbtxt'

# Path to image
PATH_TO_IMAGE_FOLDER = name + '/'

# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


# Running the actual classification

In [53]:
# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
counter = 0

for filename in os.listdir(PATH_TO_IMAGE_FOLDER):
    if filename.endswith(".jpg"):
        image_read = cv2.imread(os.path.join(PATH_TO_IMAGE_FOLDER, filename))
        image = cv2.cvtColor(image_read, cv2.COLOR_BGR2RGB)
        image_expanded = np.expand_dims(image, axis=0)
        
        # Perform the actual detection by running the model with the image as input
        (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections],feed_dict={image_tensor: image_expanded})

        
        coords = reduceBoxes(boxes[0], scores, classes)
        ranges = [findRange(coords[i]) for i in range(len(coords))]
        print(sum(ranges) / len(ranges)) if len(ranges) else print("None found")        
        # Draw the results of the detection (aka 'visulaize the results')
        vis_util.visualize_boxes_and_labels_on_image_array(
            image,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=5,
            min_score_thresh=0.50)

        # All the results have been drawn on image. Now display the image.
        image_og = cv2.cvtColor(image_read, cv2.COLOR_BGR2RGB)

        for box in coords:
            cropped = cropImg(image_og, box['box'])
            cropped.save(croppedFolder + "cropped%d.jpg" % counter)
            counter += 1

-0.13350991904735565
0.011214375495910645
-0.08316458761692047
0.006154149770736694
-0.2876562476158142
-0.021099984645843506
None found
0.1400371789932251
0.054271578788757324
-0.031650006771087646
0.33864784240722656
-0.04517360031604767
0.08665430545806885
0.11405491828918457
0.08855898678302765
-0.11229652166366577
0.17562580108642578
-0.05099807679653168
0.003516674041748047
0.21931803226470947
-0.10174649953842163
-0.17360135912895203
0.4140193462371826
0.13239893317222595
0.06745903193950653
None found
-0.10811081528663635
-0.004277825355529785
-0.1168266087770462
0.14003720879554749
0.2446954846382141
-0.13691315054893494
-0.17633140087127686
0.024616718292236328
0.1800743043422699
-0.0982297956943512
-0.20238347351551056
-0.44151225686073303
-0.0017583668231964111
None found
0.17549479007720947
-0.08746293187141418
0.0017583370208740234
-0.07330852746963501
0.09110283851623535
0.2281096875667572
-0.03141319751739502
0.21301119029521942
-0.2915901839733124
None found
-0.0332898

0.09571076929569244
-0.28238116204738617
0.0
-0.11229652166366577
-0.19645965099334717
-0.15425963699817657
0.045479923486709595
0.29117274284362793
None found
0.3008437305688858
0.17117726802825928
-0.07009649276733398
-0.3265852630138397
-0.06300678849220276
-0.34448572993278503
-0.007033318281173706
0.08064651489257812
-0.12108814716339111
-0.10350480675697327
-0.10877984762191772
None found
0.24569296836853027
-0.11211982369422913
-0.09999996423721313
-0.08416318893432617
-0.09647145867347717
-0.25600624084472656
0.08152565360069275
0.15338289737701416
-0.03868332505226135
-0.268314465880394
-0.08431793749332428
-0.03241074085235596
0.08943815529346466
0.09043572843074799
0.12272806465625763
0.03771999478340149
-0.14649516344070435
0.3981943130493164
0.14746323227882385
-0.1077822595834732
0.2070096731185913
-0.06854379177093506
0.1087799072265625
0.07009652256965637
-0.031650006771087646
None found
-0.002131342887878418
-0.08382478356361389
0.20173464715480804
0.11334574222564697
